In [1]:
import os
from typing import TypedDict, List, Annotated, Optional
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from tavily import TavilyClient
import json
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup

# Load environment variables
load_dotenv()

# Verify API keys and config are loaded
print("✓ Environment loaded")
print(f"✓ GROQ_API_KEY: {'Set' if os.getenv('GROQ_API_KEY') else 'NOT SET'}")
print(f"✓ TAVILY_API_KEY: {'Set' if os.getenv('TAVILY_API_KEY') else 'NOT SET'}")
print(f"✓ LANGSMITH_API_KEY: {'Set' if os.getenv('LANGSMITH_API_KEY') else 'NOT SET'}")
print(f"✓ LANGSMITH_TRACING: {os.getenv('LANGSMITH_TRACING')}")
print(f"✓ LANGSMITH_PROJECT: {os.getenv('LANGSMITH_PROJECT')}")

✓ Environment loaded
✓ GROQ_API_KEY: Set
✓ TAVILY_API_KEY: Set
✓ LANGSMITH_API_KEY: Set
✓ LANGSMITH_TRACING: true
✓ LANGSMITH_PROJECT: pr-overcooked-baggage-31


In [2]:
# Pydantic models for structured outputs

class Summary(BaseModel):
    """Neutral summary of the article"""
    summary: str = Field(description="A short, neutral summary of the article (2-3 sentences)")
    word_count: int = Field(description="Approximate word count of original article")

class Claim(BaseModel):
    """Individual claim or statement"""
    text: str = Field(description="The claim or statement")
    type: str = Field(description="Either 'fact' or 'opinion'")
    confidence: float = Field(description="Confidence in classification (0-1)")

class ClaimsExtraction(BaseModel):
    """Extracted claims from article"""
    factual_claims: List[Claim] = Field(description="List of factual claims")
    opinions: List[Claim] = Field(description="List of opinions")
    total_claims: int = Field(description="Total number of claims extracted")

class FactCheck(BaseModel):
    """Fact check result for a claim"""
    claim: str = Field(description="The original claim")
    status: str = Field(description="Either 'supported', 'contradicted', or 'unclear'")
    evidence: str = Field(description="Summary of evidence found")
    sources: List[str] = Field(description="URLs of sources")
    confidence: float = Field(description="Confidence in fact check (0-1)")

class FactCheckResults(BaseModel):
    """All fact check results"""
    checks: List[FactCheck] = Field(description="List of fact check results")
    needs_review: bool = Field(description="Whether human review is needed")

class LanguageAnalysis(BaseModel):
    """Analysis of language bias"""
    loaded_phrases: List[str] = Field(description="Emotionally loaded or biased phrases")
    tone: str = Field(description="Overall tone: neutral, positive, negative, inflammatory")
    language_bias_score: float = Field(description="Language bias score (0-1, 0=neutral)")
    examples: List[str] = Field(description="Example sentences showing bias")

class BiasReport(BaseModel):
    """Final bias analysis report"""
    bias_score: float = Field(description="Overall bias score (0-1, 0=unbiased)")
    stance: str = Field(description="Predicted stance or position")
    confidence: float = Field(description="Confidence in assessment (0-1)")
    key_factors: List[str] = Field(description="Key factors contributing to bias score")
    recommendation: str = Field(description="Recommendation for readers")

print("✓ Pydantic models defined")

✓ Pydantic models defined


In [3]:
# LangGraph State
class BiasDetectorState(TypedDict):
    # Input
    article_text: str
    article_url: Optional[str]
    
    # Processing stages
    summary: Optional[Summary]
    claims: Optional[ClaimsExtraction]
    fact_checks: Optional[FactCheckResults]
    language_analysis: Optional[LanguageAnalysis]
    bias_report: Optional[BiasReport]
    
    # Control flow
    needs_human_review: bool
    review_reason: Optional[str]
    human_approved: bool
    
    # Messages for tracing
    messages: List[str]

print("✓ State schema defined")

✓ State schema defined


In [4]:
# Initialize LLM with structured output support
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

# Initialize Tavily client for web search
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

print("✓ LLM initialized (Groq - llama-3.3-70b-versatile)")
print("✓ Tavily client initialized")

✓ LLM initialized (Groq - llama-3.3-70b-versatile)
✓ Tavily client initialized


In [5]:
def fetch_article_from_url(url: str) -> str:
    """
    Fetch article text from a URL using web scraping.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style", "nav", "header", "footer"]):
            script.decompose()
        
        # Get text from paragraphs
        paragraphs = soup.find_all('p')
        article_text = '\n\n'.join([p.get_text().strip() for p in paragraphs if p.get_text().strip()])
        
        if len(article_text) < 100:
            # Fallback to all text
            article_text = soup.get_text(separator='\n', strip=True)
        
        return article_text
    
    except Exception as e:
        return f"Error fetching article: {str(e)}"

print("✓ Article fetcher defined")

✓ Article fetcher defined


In [6]:
def url_input_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Node that waits for URL input via breakpoint.
    """
    print("\n📰 URL Input Node - Ready for URL")
    
    # If no URL provided yet, just wait
    if not state.get('article_url') or state['article_url'] == "":
        print("⏸️  PAUSED: Waiting for URL input...")
        return state
    
    # URL provided, fetch the article
    print(f"\n📥 Fetching article from: {state['article_url']}")
    article_text = fetch_article_from_url(state['article_url'])
    
    if article_text.startswith("Error"):
        print(f"❌ {article_text}")
        state['article_text'] = ""
        state['messages'].append(f"Failed to fetch URL")
    else:
        state['article_text'] = article_text
        state['messages'].append(f"Fetched {len(article_text)} characters")
        print(f"✓ Fetched {len(article_text)} characters")
    
    return state

print("✓ URL input node defined")

✓ URL input node defined


In [7]:
def summarize_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Generate a neutral summary of the article.
    """
    print("\n🔄 Running: Summarization Node")
    
    structured_llm = llm.with_structured_output(Summary)
    
    prompt = f"""You are a neutral news analyst. Read the following article and provide a short, neutral summary.
Focus on the main facts and events without adding interpretation or opinion.

Article:
{state['article_text'][:3000]}

Provide a 2-3 sentence neutral summary and estimate the word count."""

    summary = structured_llm.invoke([HumanMessage(content=prompt)])
    
    state['summary'] = summary
    state['messages'].append(f"Summary created: {len(summary.summary)} chars")
    
    print(f"✓ Summary: {summary.summary[:100]}...")
    
    return state

print("✓ Summarization node defined")

✓ Summarization node defined


In [8]:
def extract_claims_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Extract factual claims and opinions from the article.
    """
    print("\n🔄 Running: Claims Extraction Node")
    
    structured_llm = llm.with_structured_output(ClaimsExtraction)
    
    prompt = f"""You are a critical analyst. Read this article and extract key claims.

Separate them into:
1. FACTUAL CLAIMS: Statements that can be verified (dates, events, statistics, quotes)
2. OPINIONS: Judgments, interpretations, predictions, or subjective statements

Article:
{state['article_text'][:4000]}

For each claim, provide:
- The exact text of the claim
- Type: 'fact' or 'opinion'
- Confidence: How confident you are in the classification (0-1)

Extract 5-10 of the most important claims."""

    claims = structured_llm.invoke([HumanMessage(content=prompt)])
    
    state['claims'] = claims
    state['messages'].append(f"Extracted {claims.total_claims} claims: {len(claims.factual_claims)} facts, {len(claims.opinions)} opinions")
    
    print(f"✓ Extracted {len(claims.factual_claims)} factual claims")
    print(f"✓ Extracted {len(claims.opinions)} opinions")
    
    return state

print("✓ Claims extraction node defined")

✓ Claims extraction node defined


In [9]:
def fact_check_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Fact-check claims using Tavily web search.
    """
    print("\n🔄 Running: Fact Checking Node")
    
    claims = state['claims']
    fact_checks = []
    needs_review = False
    
    # Select top 3-5 most important factual claims
    factual_claims = claims.factual_claims[:5]
    
    for claim_obj in factual_claims:
        claim_text = claim_obj.text
        print(f"  🔍 Checking: {claim_text[:60]}...")
        
        try:
            # Search for information about the claim
            search_results = tavily_client.search(
                query=claim_text,
                max_results=3
            )
            
            # Analyze results with LLM
            context = "\n\n".join([
                f"Source: {r.get('url', 'Unknown')}\n{r.get('content', '')}" 
                for r in search_results.get('results', [])
            ])
            
            analysis_prompt = f"""Based on the following web search results, determine if this claim is supported, contradicted, or unclear.

Claim: {claim_text}

Search Results:
{context[:2000]}

Determine:
1. Status: 'supported', 'contradicted', or 'unclear'
2. Brief evidence summary
3. Confidence (0-1)

Be conservative: if evidence is mixed or insufficient, mark as 'unclear'."""

            response = llm.invoke([HumanMessage(content=analysis_prompt)])
            
            # Parse response
            status = "unclear"
            if "supported" in response.content.lower():
                status = "supported"
            elif "contradicted" in response.content.lower():
                status = "contradicted"
            
            # Extract confidence
            confidence = 0.5
            if "high confidence" in response.content.lower() or "clearly" in response.content.lower():
                confidence = 0.8
            elif "unclear" in status or "insufficient" in response.content.lower():
                confidence = 0.3
                needs_review = True
            
            fact_check = FactCheck(
                claim=claim_text,
                status=status,
                evidence=response.content[:300],
                sources=[r.get('url', '') for r in search_results.get('results', [])[:3]],
                confidence=confidence
            )
            
            fact_checks.append(fact_check)
            print(f"    ✓ Status: {status} (confidence: {confidence:.2f})")
            
        except Exception as e:
            print(f"    ✗ Error: {str(e)}")
            fact_checks.append(FactCheck(
                claim=claim_text,
                status="unclear",
                evidence=f"Error during fact check: {str(e)}",
                sources=[],
                confidence=0.0
            ))
            needs_review = True
    
    state['fact_checks'] = FactCheckResults(
        checks=fact_checks,
        needs_review=needs_review
    )
    state['needs_human_review'] = needs_review
    if needs_review:
        state['review_reason'] = "Low confidence in fact checking results"
    
    state['messages'].append(f"Fact-checked {len(fact_checks)} claims")
    
    return state

print("✓ Fact checking node defined")

✓ Fact checking node defined


In [10]:
def language_analysis_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Analyze language for emotional or loaded wording.
    """
    print("\n🔄 Running: Language Analysis Node")
    
    structured_llm = llm.with_structured_output(LanguageAnalysis)
    
    prompt = f"""You are a linguistic analyst. Analyze this article for biased or emotionally loaded language.

Look for:
- Emotionally charged words (e.g., "catastrophic", "hero", "villain")
- Loaded adjectives and adverbs
- One-sided framing
- Inflammatory rhetoric
- Persuasive language

Article excerpt:
{state['article_text'][:4000]}

Provide:
1. List of loaded phrases (with context)
2. Overall tone: neutral, positive, negative, or inflammatory
3. Language bias score (0 = perfectly neutral, 1 = extremely biased)
4. Example sentences showing bias"""

    analysis = structured_llm.invoke([HumanMessage(content=prompt)])
    
    state['language_analysis'] = analysis
    state['messages'].append(f"Language analysis: tone={analysis.tone}, bias={analysis.language_bias_score:.2f}")
    
    print(f"✓ Tone: {analysis.tone}")
    print(f"✓ Language bias score: {analysis.language_bias_score:.2f}")
    print(f"✓ Found {len(analysis.loaded_phrases)} loaded phrases")
    
    return state

print("✓ Language analysis node defined")

✓ Language analysis node defined


In [11]:
def bias_scoring_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Calculate final bias score and predict stance.
    """
    print("\n🔄 Running: Bias Scoring Node")
    
    structured_llm = llm.with_structured_output(BiasReport)
    
    # Compile all analysis
    summary_text = state['summary'].summary if state['summary'] else "No summary"
    
    fact_summary = "\n".join([
        f"- {fc.claim}: {fc.status} (conf: {fc.confidence:.2f})"
        for fc in state['fact_checks'].checks
    ]) if state['fact_checks'] else "No fact checks"
    
    lang_summary = f"Tone: {state['language_analysis'].tone}, Score: {state['language_analysis'].language_bias_score}" if state['language_analysis'] else "No analysis"
    
    prompt = f"""You are a media bias expert. Based on all the analysis, provide a final bias assessment.

SUMMARY:
{summary_text}

FACT CHECK RESULTS:
{fact_summary}

LANGUAGE ANALYSIS:
{lang_summary}
Loaded phrases: {', '.join(state['language_analysis'].loaded_phrases[:5]) if state['language_analysis'] else 'None'}

Calculate:
1. Overall bias score (0-1):
   - 0.0-0.2: Minimal bias
   - 0.2-0.4: Low bias
   - 0.4-0.6: Moderate bias
   - 0.6-0.8: High bias
   - 0.8-1.0: Extreme bias

2. Predicted stance (e.g., "pro-government", "anti-corporate", "left-leaning", "right-leaning", "neutral")

3. Confidence in your assessment (0-1)

4. Key factors contributing to the score

5. Recommendation for readers

Consider:
- Contradicted facts increase bias
- Emotional language increases bias
- One-sided coverage increases bias
- Missing context increases bias"""

    report = structured_llm.invoke([HumanMessage(content=prompt)])
    
    state['bias_report'] = report
    state['messages'].append(f"Final bias score: {report.bias_score:.2f}, stance: {report.stance}")
    
    # Check if we need review due to high bias or low confidence
    if report.bias_score > 0.7 or report.confidence < 0.5:
        state['needs_human_review'] = True
        state['review_reason'] = f"High bias score ({report.bias_score:.2f}) or low confidence ({report.confidence:.2f})"
    
    print(f"✓ Bias score: {report.bias_score:.2f}")
    print(f"✓ Stance: {report.stance}")
    print(f"✓ Confidence: {report.confidence:.2f}")
    
    return state

print("✓ Bias scoring node defined")

✓ Bias scoring node defined


In [12]:
def human_review_node(state: BiasDetectorState) -> BiasDetectorState:
    """
    Pause for human review if needed.
    """
    print("\n⚠️  HUMAN REVIEW REQUIRED")
    print(f"Reason: {state.get('review_reason', 'Unknown')}")
    print("\nCurrent Analysis:")
    print(f"- Bias Score: {state['bias_report'].bias_score:.2f}")
    print(f"- Stance: {state['bias_report'].stance}")
    print(f"- Confidence: {state['bias_report'].confidence:.2f}")
    
    if state['fact_checks']:
        print(f"\nFact Checks:")
        for fc in state['fact_checks'].checks:
            print(f"  - {fc.claim[:60]}... → {fc.status}")
    
    print("\n✓ Review node reached")
    
    return state

print("✓ Human review node defined")

✓ Human review node defined


In [13]:
def should_review(state: BiasDetectorState) -> str:
    """
    Decide whether to route to human review or final output.
    """
    if state.get('needs_human_review', False) and not state.get('human_approved', False):
        return "review"
    return "end"

print("✓ Routing functions defined")

✓ Routing functions defined


In [14]:
# Build the LangGraph workflow with URL input breakpoint
workflow = StateGraph(BiasDetectorState)

# Add nodes
workflow.add_node("url_input", url_input_node)  # NEW: First node for URL input
workflow.add_node("summarize", summarize_node)
workflow.add_node("extract_claims", extract_claims_node)
workflow.add_node("fact_check", fact_check_node)
workflow.add_node("language_analysis", language_analysis_node)
workflow.add_node("bias_scoring", bias_scoring_node)
workflow.add_node("human_review", human_review_node)

# Define edges - Start with URL input
workflow.set_entry_point("url_input")  # CHANGED: Start with URL input node
workflow.add_edge("url_input", "summarize")  # NEW: url_input -> summarize
workflow.add_edge("summarize", "extract_claims")
workflow.add_edge("extract_claims", "fact_check")
workflow.add_edge("fact_check", "language_analysis")
workflow.add_edge("language_analysis", "bias_scoring")

# Conditional edge for human review
workflow.add_conditional_edges(
    "bias_scoring",
    should_review,
    {
        "review": "human_review",
        "end": END
    }
)

workflow.add_edge("human_review", END)

# Compile with checkpointer - TWO breakpoints now
memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory, 
    interrupt_before=["url_input", "human_review"]  # CHANGED: Two breakpoints
)

print("✓ LangGraph workflow compiled with URL input breakpoint")
print("✓ Breakpoints: url_input (for URL), human_review (if needed)")

✓ LangGraph workflow compiled with URL input breakpoint
✓ Breakpoints: url_input (for URL), human_review (if needed)


In [15]:
def display_results(state: BiasDetectorState):
    """
    Pretty print the analysis results.
    """
    print("\n" + "="*80)
    print("📰 NEWS BIAS DETECTION REPORT")
    print("="*80)
    
    if state.get('article_url'):
        print(f"\n🔗 Source URL: {state['article_url']}")
    
    # Summary
    if state.get('summary'):
        print(f"\n📝 SUMMARY:")
        print(f"{state['summary'].summary}")
        print(f"Word count: ~{state['summary'].word_count}")
    
    # Claims
    if state.get('claims'):
        print(f"\n📊 CLAIMS EXTRACTED:")
        print(f"  Factual claims: {len(state['claims'].factual_claims)}")
        print(f"  Opinions: {len(state['claims'].opinions)}")
        
        print("\n  Top Factual Claims:")
        for i, claim in enumerate(state['claims'].factual_claims[:3], 1):
            print(f"    {i}. {claim.text}")
        
        print("\n  Top Opinions:")
        for i, opinion in enumerate(state['claims'].opinions[:3], 1):
            print(f"    {i}. {opinion.text}")
    
    # Fact Checks
    if state.get('fact_checks'):
        print(f"\n🔍 FACT CHECK RESULTS:")
        for fc in state['fact_checks'].checks:
            print(f"\n  Claim: {fc.claim}")
            print(f"  Status: {fc.status.upper()}")
            print(f"  Confidence: {fc.confidence:.2f}")
            if fc.sources:
                print(f"  Sources: {fc.sources[0]}")
    
    # Language Analysis
    if state.get('language_analysis'):
        lang = state['language_analysis']
        print(f"\n💬 LANGUAGE ANALYSIS:")
        print(f"  Tone: {lang.tone}")
        print(f"  Language Bias Score: {lang.language_bias_score:.2f}")
        print(f"  Loaded Phrases: {len(lang.loaded_phrases)}")
        if lang.loaded_phrases:
            print(f"    Examples: {', '.join(lang.loaded_phrases[:3])}")
    
    # Final Report
    if state.get('bias_report'):
        report = state['bias_report']
        print(f"\n⚖️  FINAL BIAS ASSESSMENT:")
        print(f"  Bias Score: {report.bias_score:.2f} / 1.0")
        print(f"  Stance: {report.stance}")
        print(f"  Confidence: {report.confidence:.2f}")
        
        print(f"\n  Key Factors:")
        for factor in report.key_factors:
            print(f"    • {factor}")
        
        print(f"\n  💡 Recommendation:")
        print(f"  {report.recommendation}")
    
    print("\n" + "="*80)

print("✓ Display function defined")

✓ Display function defined


In [16]:
def analyze_with_url_breakpoint(thread_id: str = "url_analysis"):
    """
    Start analysis with URL input breakpoint.
    This is the main function to use for interactive URL input.
    """
    print("🚀 Starting News Bias Detector with URL Input")
    print("=" * 80)
    
    # Step 1: Initialize empty state
    initial_state = {
        "article_text": "",
        "article_url": "",  # Empty - will be filled at breakpoint
        "summary": None,
        "claims": None,
        "fact_checks": None,
        "language_analysis": None,
        "bias_report": None,
        "needs_human_review": False,
        "review_reason": None,
        "human_approved": False,
        "messages": []
    }
    
    config = {"configurable": {"thread_id": thread_id}}
    
    # Step 2: Run until url_input breakpoint
    print("\n⏸️  Workflow will pause for URL input...")
    for state in app.stream(initial_state, config):
        pass
    
    # Step 3: Get URL from user
    url = input("\n📰 Enter news article URL: ").strip()
    
    if not url:
        print("❌ No URL provided")
        return None
    
    # Step 4: Update state with URL
    current_state = app.get_state(config)
    current_values = current_state.values
    current_values['article_url'] = url
    app.update_state(config, current_values)
    
    print(f"\n✓ URL set: {url}")
    print("▶️  Continuing analysis...\n")
    
    # Step 5: Continue workflow
    final_state = None
    for state in app.stream(None, config):
        final_state = list(state.values())[0]
        
        # Check for human review breakpoint
        if final_state.get('needs_human_review') and not final_state.get('human_approved'):
            print("\n⏸️  Paused at human review")
            approve = input("\n❓ Approve and continue? (yes/no): ").strip().lower()
            
            if approve == 'yes':
                current_state = app.get_state(config)
                current_values = current_state.values
                current_values['human_approved'] = True
                app.update_state(config, current_values)
                
                for state in app.stream(None, config):
                    final_state = list(state.values())[0]
            else:
                print("❌ Analysis stopped")
                return final_state
    
    # Step 6: Display and export results
    if final_state:
        display_results(final_state)
        
        # Auto-generate filename from thread_id
        filename = f"analysis_{thread_id}.json"
        export_results(final_state, filename)
    
    return final_state

print("✓ URL breakpoint analysis function defined")

✓ URL breakpoint analysis function defined


In [17]:
def export_results(state: BiasDetectorState, filename: str = "bias_report.json"):
    """
    Export analysis results to JSON file.
    """
    output = {
        "article_url": state.get('article_url'),
        "summary": {
            "text": state['summary'].summary if state.get('summary') else None,
            "word_count": state['summary'].word_count if state.get('summary') else None
        },
        "claims": {
            "factual": [{"text": c.text, "confidence": c.confidence} 
                       for c in state['claims'].factual_claims] if state.get('claims') else [],
            "opinions": [{"text": c.text, "confidence": c.confidence} 
                        for c in state['claims'].opinions] if state.get('claims') else []
        },
        "fact_checks": [
            {
                "claim": fc.claim,
                "status": fc.status,
                "confidence": fc.confidence,
                "sources": fc.sources
            } for fc in state['fact_checks'].checks
        ] if state.get('fact_checks') else [],
        "language_analysis": {
            "tone": state['language_analysis'].tone if state.get('language_analysis') else None,
            "bias_score": state['language_analysis'].language_bias_score if state.get('language_analysis') else None,
            "loaded_phrases": state['language_analysis'].loaded_phrases if state.get('language_analysis') else []
        },
        "bias_report": {
            "bias_score": state['bias_report'].bias_score if state.get('bias_report') else None,
            "stance": state['bias_report'].stance if state.get('bias_report') else None,
            "confidence": state['bias_report'].confidence if state.get('bias_report') else None,
            "key_factors": state['bias_report'].key_factors if state.get('bias_report') else [],
            "recommendation": state['bias_report'].recommendation if state.get('bias_report') else None
        }
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=2, ensure_ascii=False)
    
    print(f"✓ Results exported to {filename}")

print("✓ Export function defined")

✓ Export function defined


In [18]:
# Main usage: Analyze with URL input breakpoint
# Run this cell to start - it will ask for URL input

result = analyze_with_url_breakpoint(thread_id="my_analysis")

🚀 Starting News Bias Detector with URL Input

⏸️  Workflow will pause for URL input...

✓ URL set: https://thewire.in/history/past-continuous-1993-bombay-blasts-were-in-response-an-entire-community-being-alienated
▶️  Continuing analysis...


📰 URL Input Node - Ready for URL

📥 Fetching article from: https://thewire.in/history/past-continuous-1993-bombay-blasts-were-in-response-an-entire-community-being-alienated
✓ Fetched 208 characters

🔄 Running: Summarization Node
✓ Summary: The 1993 Bombay blasts were part of a larger pattern of terrorist attacks. The blasts occurred in 19...

🔄 Running: Claims Extraction Node
✓ Extracted 2 factual claims
✓ Extracted 1 opinions

🔄 Running: Fact Checking Node
  🔍 Checking: 1993 Bombay Blasts...
    ✓ Status: supported (confidence: 0.80)
  🔍 Checking: The Wire News India, Latest News,News from India, Politics, ...
    ✓ Status: supported (confidence: 0.80)

🔄 Running: Language Analysis Node
✓ Tone: negative
✓ Language bias score: 0.60
✓ Found 2 load

In [19]:
# Display complete bias report from JSON
import json

try:
    with open('analysis_my_analysis.json', 'r', encoding='utf-8') as f:
        saved_results = json.load(f)
    
    print("\n" + "="*80)
    print("📊 COMPLETE BIAS ANALYSIS SUMMARY")
    print("="*80)
    
    bias = saved_results.get('bias_report', {})
    print(f"\n⚖️  Bias Score: {bias.get('bias_score', 'N/A')} / 1.0")
    print(f"📍 Stance: {bias.get('stance', 'N/A')}")
    print(f"🎯 Confidence: {bias.get('confidence', 'N/A')}")
    
    print(f"\n🔑 Key Factors:")
    for factor in bias.get('key_factors', []):
        print(f"   • {factor}")
    
    print(f"\n💡 Recommendation:")
    print(f"   {bias.get('recommendation', 'N/A')}")
    
    lang = saved_results.get('language_analysis', {})
    print(f"\n💬 Language Analysis:")
    print(f"   Tone: {lang.get('tone', 'N/A')}")
    print(f"   Language Bias: {lang.get('bias_score', 'N/A')}")
    
    print("\n" + "="*80)
    
except FileNotFoundError:
    print("❌ JSON file not found - run analysis first")


📊 COMPLETE BIAS ANALYSIS SUMMARY

⚖️  Bias Score: 0.5 / 1.0
📍 Stance: left-leaning
🎯 Confidence: 0.8

🔑 Key Factors:
   • negative tone
   • loaded phrases
   • one-sided coverage

💡 Recommendation:
   Readers should consider multiple sources for a balanced view

💬 Language Analysis:
   Tone: negative
   Language Bias: 0.6

